In [235]:
import pandas as pd
import os
import glob

In [236]:
os.getcwd()

'/vortexfs1/omics/env-bio/collaboration/dinoflagellates_METZYME/data/metaT_trimmed_reads/fasta_files/paired/mRNA'

In [246]:
os.chdir('/vortexfs1/omics/env-bio/collaboration/dinoflagellates_METZYME/data/metaT_trimmed_reads/' \
            'fasta_files/paired/mRNA')
taxon_380m = '30B91_S28_001_380m_mRNA_diamond_faa_out.tsv'

In [247]:
full_col_names = ['Query ID', 'Subject ID', 'Percentage of identical matches', 'Alignment length', 
             'Number of mismatches', 'Number of gap openings', 'Start of alignment in query', 
             'End of alignment in query', 'Start of alignment in subject', 'End of alignment in subject',
             'Expected value', 'Bit score']

short_col_names = ['qseqid', 'sseqid', 'pident', 'length', 'mismatch',
'gapopen', 'qstart', 'qend', 'sstart', 'send', 'evalue', 'bitscore']

df_380m = pd.read_csv(taxon_380m, sep='\t', names=short_col_names)

In [248]:
df_380m.head(20)

,qseqid,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore
0,NS500647:110:HMV77BGXX:1:11101:11808:1124_2_139_-,Karlodinium-micrum-CCMP2283-20140214|83450_1,73.2,41,11,0,1,41,262,302,8.600000e-09,65.5
1,NS500647:110:HMV77BGXX:1:11101:15804:1143_3_140_+,Karlodinium-micrum-CCMP2283-20140214|13279_1,73.0,37,10,0,8,44,245,281,6.600000e-09,65.9
2,NS500647:110:HMV77BGXX:1:11101:10354:1104_1_135_+,gi119469189-NZ_AAVS01000005,96.7,30,1,0,15,44,1,30,1.600000e-07,61.2
3,NS500647:110:HMV77BGXX:1:11101:6355:1145_3_134_-,gi494242305-NZ_AFIG01000001,88.6,44,5,0,1,44,360,403,1.600000e-12,77.8
4,NS500647:110:HMV77BGXX:1:11101:5599:1113_2_142_+,gi497816202-NZ_AFPJ01000007,93.6,47,3,0,1,47,119,165,8.200000e-15,85.5
5,NS500647:110:HMV77BGXX:1:11101:1920:1114_1_138_-,Karenia-brevis-CCMP2229-20130916|2192_1,78.3,46,10,0,1,46,534,579,1.500000e-13,81.3
6,NS500647:110:HMV77BGXX:1:11101:11808:1124_2_142_+,Karlodinium-micrum-CCMP2283-20140214|83450_1,80.9,47,8,1,1,47,201,246,2.100000e-10,70.9
7,NS500647:110:HMV77BGXX:1:11101:6355:1145_2_130_+,Csal_2160-NC_007963,85.7,42,6,0,1,42,296,337,2.300000e-11,73.9
8,NS500647:110:HMV77BGXX:1:11101:17487:1190_2_130_-,C943_2110-GCA_000330725.1_contig18,83.7,43,7,0,1,43,859,901,6.600000e-11,72.4
9,NS500647:110:HMV77BGXX:1:11101:21960:1162_3_137_-,MMETSP1335-20130426|36080_1,52.8,36,17,0,10,45,17,52,4.800000e-04,49.7


In [249]:
set(df_380m['sseqid'])

{'amad1_13795-NC_019393',
 'MMETSP1050-20130122|23413_1',
 'gi498123623-NZ_AEYW01000002',
 'CA2559_07435-NC_014230',
 'HMPREF6485_1248-NZ_AEPD01000026',
 'Karenia-brevis-SP1-20130916|291799_1',
 'PCA10_38340-NC_021499',
 'VIBR0546_15197-NZ_AEVS01000103',
 'Lingulodinium-polyedra-CCMP1738-20130920|23854_1',
 'EDI_076220-edi5415',
 'P9301_07541-NC_009091',
 'Pedsa_0438-NC_015177',
 'Scrippsiella-hangoei-SHTV5-20131105|56738_1',
 'NIDE1728-NC_014355',
 'Glenodinium-foliaceum-CCAP1116_3-20130913|266753_1',
 'Undescribed-sp-D1-20130905|215330_1',
 'Ornrh_1552-Ornrh_Contig157.1',
 'Aeqsu_1911-Aeqsu_Contig157.1',
 'MMETSP0208-20121228|12957_1',
 'gi497782718-NZ_AEWH01000045',
 'MMETSP0448-20130528|7503_1',
 'Thalassiosira-miniscula-CCMP1093-20140214|11900_1',
 'MMETSP0689_2-20121128|6884_1',
 'MMETSP0472-20121206|42291_1',
 'gi161325954-ABIB01000002',
 'gi494854135-NZ_BADV01000009',
 '100120193-nvi12072',
 'W5A_06965-GCA_000260835.1_Contig6',
 'MMETSP0224-20130122|16785_1',
 'DappuP300571-sca

In [ ]:
#!/usr/bin/env python3

from Bio import SeqIO
import csv
import sys

'''
Annotates Trinity fasta outputs with phylodb hits using DIAMOND tabular output
Usage:
    python3 fastqannotation.py <trinitycontigs.fasta> <diamondoutput.m8> <annotatedout.fasta> <mappingfile.tsv>
'''

# PhyloDB paths can be soft-coded if you comment out the file paths and uncomment the next two lines
# If this is done, the taxonomy and gene file will need to be supplied to the program call

taxonomy_file = "/proj/marchlab/projects/phylodbannotation/phylodb_1.076.taxonomy.txt"
gene_file = "/proj/marchlab/projects/phylodbannotation/phylodb_1.076.annotations.txt"
#taxonomy_file = sys.argv[5]
#annotated_file = sys.argv[6]

# Get data files from command line
contigs_file = sys.argv[1]
diamond_file = sys.argv[2]
annotated_file = sys.argv[3]
mapping_file = sys.argv[4]

# Load phylodb master-lists as dictionaries for easy parsing
diamond_dict = {}
tax_dict = {}
gene_dict = {}

with open(diamond_file) as diamondfile:
    for line in diamondfile:
        row = line.strip().split("\t")
        diamond_dict[row[0]] = row[1:]

with open(taxonomy_file) as taxfile:
    for line in taxfile:
        row = line.strip().split("\t")
        tax_dict[row[0]] = row[1:]

with open(gene_file) as genefile:
    for line in genefile:
        row = line.strip().split("\t")
        gene_dict[row[0]] = row[1:]

'''
1. Get each fasta descriptor
2. If descriptor is in diamond output, get diamond annotation
3. Use diamond annotation to get gene annotation from gene_dict
4. Use gene annotation as key for tax_dict
5. Write all fields in fasta descriptor as new header
'''




with open(contigs_file, 'rU') as contigs, open(annotated_file, 'w') as annotated, open(mapping_file, 'w') as mapping:
    records = SeqIO.parse(contigs, "fasta")
    writer = csv.writer(mapping, delimiter='\t')
    writer.writerow(["TrinityID", "Gene", "Organism", "Taxonomy"])
    for record in records:  
        if record.id in diamond_dict:
            
            old_record = record.id

            gene = diamond_dict[record.id][0]

            organism = gene_dict[gene][1]

            tax = tax_dict[organism][1]

            record.id = "\t".join([old_record, str(gene), organism, str(tax)])
            SeqIO.write(record, annotated, 'fasta-2line')

            writer.writerow([old_record, gene, organism, tax])

In [250]:
### Modified from Logan Whitehouse's lab, 
# https://github.com/Lswhiteh/phylodbannotation/blob/master/fastaannotation.py

taxonomy_file = "/vortexfs1/omics/env-bio/collaboration/dinoflagellates_METZYME/databases/PhyloDB/phylodb_1.076.taxonomy.txt"
gene_file = "/vortexfs1/omics/env-bio/collaboration/dinoflagellates_METZYME/databases/PhyloDB/phylodb_1.076.annotations.txt"

tax_dict = {}
gene_dict = {}

with open(taxonomy_file) as taxfile:
    for line in taxfile:
        row = line.strip().split("\t")
        tax_dict[row[0]] = row[1:]

with open(gene_file) as genefile:
    for line in genefile:
        row = line.strip().split("\t")
        gene_dict[row[0]] = row[1:]

In [254]:
gene_mapping = df_380m.head()['sseqid'].map(gene_dict)
## Massage data to get just the second column (organism classification)
organism_df = pd.DataFrame(gene_mapping.values.tolist())[1]
taxonomy_mapping = organism_df.map(tax_dict)
tax_df = pd.DataFrame(taxonomy_mapping.values.tolist())[1]
#splitting up taxonomy into individual groupings
tax_df = pd.DataFrame(tax_df.str.split(';').tolist())
tax_df.columns = ['Domain', 'Kingdom', 'Phylum', 'Class', 'Order', 'Family', 'Genus', 'Strain_name']
df_380m_tax = pd.concat([df_380m, tax_df], axis=1)

In [267]:
df_380m_tax.to_csv(dm_file+'_taxonomy.tsv', sep='\t')

In [270]:
df_380m_tax[df_380m_tax['Phylum'] == 'Dinophyta']

,qseqid,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore,Domain,Kingdom,Phylum,Class,Order,Family,Genus,Strain_name
0,NS500647:110:HMV77BGXX:1:11101:11808:1124_2_139_-,Karlodinium-micrum-CCMP2283-20140214|83450_1,73.2,41,11,0,1,41,262,302,8.600000e-09,65.5,Eukaryota,Alveolata,Dinophyta,Dinophyceae,Dinophyceae_X,Dinophyceae_XX,Karlodinium,"Karlodinium micrum, Strain CCMP2283"
1,NS500647:110:HMV77BGXX:1:11101:15804:1143_3_140_+,Karlodinium-micrum-CCMP2283-20140214|13279_1,73.0,37,10,0,8,44,245,281,6.600000e-09,65.9,Eukaryota,Alveolata,Dinophyta,Dinophyceae,Dinophyceae_X,Dinophyceae_XX,Karlodinium,"Karlodinium micrum, Strain CCMP2283"


In [23]:
phylodb_tax = glob.glob('/vortexfs1/omics/env-bio/collaboration/dinoflagellates_METZYME/'\
                        'databases/PhyloDB/*taxonomy*')[0]

In [27]:
df_phylodb = pd.read_csv(phylodb_tax, sep="\t", compression='gzip')

In [68]:
phylodb_pep = glob.glob('/vortexfs1/omics/env-bio/collaboration/dinoflagellates_METZYME/'\
                        'databases/PhyloDB/*pep*')[0]

In [72]:
df = pd.read_csv(phylodb_pep, sep="\t", compression='gzip')